In [ ]:
import re
from collections import defaultdict
import numpy as np
import pandas as pd

Функция по очистке текста от знаков препинания:

In [ ]:
punct = "!\"#$%&'()*+,-./:;<=>?@[\\]^_{|}~—"

def clear_text(text):
    text = text.lower()
    return text.translate(str.maketrans("", "", punct)).replace("\n", " ").strip()

Текст для анализа с расставленными ударениями:

In [ ]:
# М. Лермонтов «Белеет парус одинокий»

text = """
Беле`ет па`рус одино`кий
В тума`не мо`ря голубо`м!..
Что` ище`т о`н в стране` далё`кой?
Что` ки`нул о`н в краю` родно`м?...

Игра`ют во`лны — ве`тер сви`щет,
И ма`чта гнё`тся и скрипи`т...
Увы`! О`н сча`стия не ище`т
И не от сча`стия бежи`т!

Под ни`м струя` светле`й лазу`ри,
Над ни`м лу`ч со`лнца золото`й...
А о`н, мяте`жный, про`сит бу`ри,
Ка`к будто` в бу`рях е`сть поко`й!
"""
text = clear_text(text)
text

'беле`ет па`рус одино`кий в тума`не мо`ря голубо`м что` ище`т о`н в стране` далё`кой что` ки`нул о`н в краю` родно`м  игра`ют во`лны  ве`тер сви`щет и ма`чта гнё`тся и скрипи`т увы` о`н сча`стия не ище`т и не от сча`стия бежи`т  под ни`м струя` светле`й лазу`ри над ни`м лу`ч со`лнца золото`й а о`н мяте`жный про`сит бу`ри ка`к будто` в бу`рях е`сть поко`й'

Подсчитываем количество звуков.

Правила подсчета:
1.	Ударные звуки, как более важные, учитываются в сумме с коэффициентом 2.
2.	Первый звук в слове, как более важный, учитывается в сумме с коэффициентом 4.


In [ ]:
df = defaultdict(int)
vow = "оаяеиуюыёй"
for word in text.split():
    first = True
    for i, char in enumerate(word):
        if char in vow:
            df[char] += 4 if first else 1
            first = False
        elif char == "`":
            df[word[i - 1]] += 1
df

defaultdict(int,
            {'е': 46,
             'а': 55,
             'у': 36,
             'о': 92,
             'и': 59,
             'й': 6,
             'я': 11,
             'ё': 7,
             'ю': 3,
             'ы': 4})

Общее количество звуков в тексте (суммируем):

In [ ]:
R = 0
for i in df:
  R += df[i]
R

319

Формула нормативного разброса значения нормативной частоты звука в русской речи согласно центральной предельной теореме

$\large\sigma_i = \sqrt{\frac{p_{in}*(1-p_{in})}{R}}$

где:
* $\sigma_i$ – стандартное отклонение (дисперсия),
* $p_{in}$ – норма частотности соответствующей буквы (звука), нормативные значения рассчитаны по словарю русского языка и используются всеми как константы,
* $R$ – общее число букв (звуков) в тексте.


Норма частотности букв/звуков для русского языка:

* **А+Я** (0,116),
* **О+0.5Ё** (0.126),
* **Е+0.5Ё** (0.102),
* **И+0.5Й** (0.077),
* **У+Ю** (0,040),
* **Ы** (0.024)


Просуммируем буквы под норму частотности:

In [ ]:
df["а+я"] = df["а"] + df["я"]
df["о+ё"] = df["о"] + df["ё"] / 2
df["е+ё"] = df["е"] + df["ё"] / 2
df["и+й"] = df["и"] + df["й"] / 2
df["у+ю"] = df["у"] + df["ю"]
del df["а"], df["я"], df["о"], df["е"], df["ё"], df["и"], df["й"], df["у"], df["ю"]
df

defaultdict(int,
            {'ы': 4,
             'а+я': 66,
             'о+ё': 95.5,
             'е+ё': 49.5,
             'и+й': 62.0,
             'у+ю': 39})

Формула расчета силы цвета:

$\large a_i=\frac{p_{i}-p_{in}}{\sigma_i}$

где
* $\sigma_i$ – стандартное отклонение,
* $p_{i}$– частота буквы в тексте,
* $p_{in}$ – норма частотности соответствующей буквы.



Вычислим силу цвета:

In [ ]:
df2 = pd.DataFrame({"R":df, "Pn":{'ы': 0.024, 'а+я': 0.116, 'о+ё': 0.126, 'е+ё': 0.102, 'и+й': 0.077, 'у+ю': 0.040}})
df2["Pi"] = df2["R"]/R
df2 = df2[["R", "Pi", "Pn"]]
df2["sig"] = np.sqrt(df2["Pn"]*(1-df2["Pn"])/R)
df2["a"] = (df2["Pi"] - df2["Pn"]) / df2["sig"]
df2

,R,Pi,Pn,sig,a
ы,4.0,0.012539,0.024,0.008569,-1.337459
а+я,66.0,0.206897,0.116,0.017929,5.069763
о+ё,95.5,0.299373,0.126,0.018580,9.331166
е+ё,49.5,0.155172,0.102,0.016945,3.137932
и+й,62.0,0.194357,0.077,0.014926,7.862479
у+ю,39.0,0.122257,0.040,0.010972,7.497265


Итог:

**М. Лермонтов «Белеет парус одинокий» имеет окраску:**  
ярко-красный : бело-жёлтый : жёлто-зелёный : синий  : лиловый  
5.07 : 9.33 : 3.14 : 7.86 : 7.49

чёрно-коричневый цвет Лермонтов старался убирать
